# Tritement des données

Importation des modules

In [58]:
import pandas as pd  
import numpy as np, pandas as pd, matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, chi2_contingency
from scipy.stats.contingency import association
from scipy.stats import mannwhitneyu, norm


In [ ]:

# USAGERS impliqués dans l'accident
base_us = pd.read_csv("usagers-2021.csv", delimiter=",", low_memory=False)

#VEHICULES impliqués dans l'accident
base_veh = pd.read_csv("vehicules-2021.csv", delimiter=";", encoding="ISO-8859-1", low_memory=False)

In [ ]:
# Charger le fichier qui décrit les circonstances générales de l’accident
base_car = pd.read_excel("caracteristiques.xlsx")    

In [35]:
base_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56530 entries, 0 to 56529
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Num_Acc  56530 non-null  int64 
 1   jour     56530 non-null  int64 
 2   mois     56530 non-null  int64 
 3   an       56530 non-null  int64 
 4   hrmn     56530 non-null  object
 5   lum      56530 non-null  int64 
 6   com      56530 non-null  object
 7   agg      56530 non-null  int64 
 8   int      56530 non-null  int64 
 9   atm      56530 non-null  int64 
 10  col      56530 non-null  int64 
 11  adr      55957 non-null  object
 12  lat      56530 non-null  object
 13  long     56530 non-null  object
dtypes: int64(9), object(5)
memory usage: 6.0+ MB


In [54]:
print(" la base des caracteristiques contient", sum(base_car.duplicated('Num_Acc')), "doublons")

 la base des caracteristiques contient 11 doublons


In [ ]:
#pour compter les doublon pur/parfaits
print("la base des caracteristiques contient", sum(base_car.duplicated()),'doublon purs')

la base des caracteristiques contient 11 doublon purs


In [ ]:
# Identifier les lignes dupliquées
duplicated = base_car[base_car.duplicated( keep=False)]
# Trier les lignes dupliquées par ordre croissant de 'Num_Acc'
duplicated = duplicated.sort_values(by='Num_Acc')
# Afficher les lignes dupliquées triées
duplicated 


,Num_Acc,jour,mois,an,hrmn,lum,com,agg,int,atm,col,adr,lat,long
33687,202100013498,12,10,2021,11:00,1,CC_71076,2,2,1,3,Avenue de Paris,"46,79438402","4,848719001"
6317,202100013498,12,10,2021,11:00,1,CC_71076,2,2,1,3,Avenue de Paris,"46,79438402","4,848719001"
25206,202100017917,21,9,2021,09:45,1,CC_95280,2,1,1,3,RD 47A,"49,01144865","2,479454682"
19025,202100017917,21,9,2021,09:45,1,CC_95280,2,1,1,3,RD 47A,"49,01144865","2,479454682"
32639,202100023406,19,8,2021,10:00,1,CC_97120,2,1,1,6,UNIVERSITE FOUILLOLE,"16,22706","-61,52962"
420,202100023406,19,8,2021,10:00,1,CC_97120,2,1,1,6,UNIVERSITE FOUILLOLE,"16,22706","-61,52962"
45121,202100027321,23,7,2021,11:10,1,CC_97302,2,1,1,3,A.HENRI(RUE),"4,922661236","-52,3113595"
50662,202100027321,23,7,2021,11:10,1,CC_97302,2,1,1,3,A.HENRI(RUE),"4,922661236","-52,3113595"
2994,202100027995,20,7,2021,18:20,1,CC_62307,2,1,1,1,Route de Peuplingues,"50,914111","1,724478"
55470,202100027995,20,7,2021,18:20,1,CC_62307,2,1,1,1,Route de Peuplingues,"50,914111","1,724478"


In [45]:
base_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129189 entries, 0 to 129188
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      129189 non-null  int64  
 1   id_vehicule  129189 non-null  object 
 2   num_veh      129189 non-null  object 
 3   place        129189 non-null  int64  
 4   catu         129189 non-null  int64  
 5   grav         129189 non-null  int64  
 6   sexe         129189 non-null  int64  
 7   an_nais      126122 non-null  float64
 8   trajet       129189 non-null  int64  
 9   secu1        129189 non-null  int64  
 10  secu2        129189 non-null  int64  
 11  secu3        129189 non-null  int64  
 12  locp         129189 non-null  int64  
 13  actp         129189 non-null  object 
 14  etatp        129189 non-null  int64  
dtypes: float64(1), int64(11), object(3)
memory usage: 14.8+ MB


In [39]:
base_veh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97365 entries, 0 to 97364
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ï»¿id_vehicule  97365 non-null  object 
 1   num_veh         97365 non-null  object 
 2   senc            97365 non-null  int64  
 3   catv            97365 non-null  int64  
 4   obs             97365 non-null  int64  
 5   obsm            97365 non-null  int64  
 6   choc            97365 non-null  int64  
 7   manv            97365 non-null  int64  
 8   motor           97365 non-null  int64  
 9   occutc          744 non-null    float64
dtypes: float64(1), int64(7), object(2)
memory usage: 7.4+ MB
